# Численное дифференцирование

## 1 Найти точное и приближенные значения первой производной функции индивидуального задания в точке x (конкретное, не обращающее функцию в 0 значение, выбрать самостоятельно), использовав формулы численного дифференцирования первого, второго и четвертого порядка аппроксимации, вычислив или подобрав оптимальный шаг h. Для каждого полученного значения определить абсолютную, относительную погрешности (сопоставив точное и приближенные значения), проверить теоретическую оценку абсолютной погрешности r.

In [1]:
# Установка Rust
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] += ":/root/.cargo/bin"

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2025-03-18, rust version 1.85.1 (4eb161250 2025-03-15)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
 29.2 MiB /  29.2 MiB (100 %)  18.0 MiB/s in  1s
info: downloading component 'rustc'
 69.5 MiB /  69.5 MiB (100 %)  16.9 MiB/s in  3s
info: downloading component 'rustfmt'
info: installing component 'cargo'
  8.8 MiB /   8.8 MiB (100 %)   6.1 MiB/s in  1s
info: installing component 'clippy'
info: installing component 'rust-docs'
 18.2 MiB /  18.2 MiB (100 %)   2.4 MiB/s in  8s
info: installing component 'rust-std'
 29.2 MiB /  29.2 MiB (100 %)   4.8 MiB/s in  6s
info: installing component 'rustc'
 69.5 MiB /  69.5 MiB (100 %)   5.6 MiB/s in 12s
info: installing component 'rustfmt'
i

In [2]:
# Проверка установки Rust
!rustc --version

rustc 1.85.1 (4eb161250 2025-03-15)


##  2 Найти точное и приближенное значения второй производной функции индивидуального задания в точке x. Подобрать оптимальный шаг h экспериментально. Определить абсолютную, относительную погрешности.

## 3 Найти точное и приближенное значения третьей производной функции индивидуального задания в точке x. Подобрать оптимальный шаг h экспериментально. Определить абсолютную, относительную погрешности.

## 4 Подготовить программы на языке Rust для всех численных расчетов.


In [7]:
%%writefile main.rs

use std::f64;

// Исходная функция
fn f(x: f64) -> f64 {
    x.cos() * x.sinh() + (-0.2 * x).exp() * (x + 10.0).ln()
}

// Аналитические производные
fn f_prime_exact(x: f64) -> f64 {
    -x.sin() * x.sinh() + x.cos() * x.cosh() + (-0.2 * x).exp() * (-0.2 * (x + 10.0).ln() + 1.0 / (x + 10.0))
}

fn f_double_prime_exact(x: f64) -> f64 {
    -2.0 * x.sin() * x.cosh() + (-0.2 * x).exp() * (
        0.04 * (x + 10.0).ln() - 0.4 / (x + 10.0) - 1.0 / (x + 10.0).powi(2)
    )
}

fn f_triple_prime_exact(x: f64) -> f64 {
    -2.0 * (x.cos() * x.cosh() + x.sin() * x.sinh()) + (-0.2 * x).exp() * (
        -0.2 * (0.04 * (x + 10.0).ln() - 0.4 / (x + 10.0) - 1.0 / (x + 10.0).powi(2)) +
        0.04 / (x + 10.0) + 0.4 / (x + 10.0).powi(2) + 2.0 / (x + 10.0).powi(3)
    )
}

// Численные методы
fn forward_diff<F: Fn(f64) -> f64>(f: F, x: f64, h: f64) -> f64 {
    (f(x + h) - f(x)) / h
}

fn central_diff<F: Fn(f64) -> f64>(f: F, x: f64, h: f64) -> f64 {
    (f(x + h) - f(x - h)) / (2.0 * h)
}

fn fourth_order_diff<F: Fn(f64) -> f64>(f: F, x: f64, h: f64) -> f64 {
    (f(x - 2.0 * h) - 8.0 * f(x - h) + 8.0 * f(x + h) - f(x + 2.0 * h)) / (12.0 * h)
}

fn second_derivative<F: Fn(f64) -> f64>(f: F, x: f64, h: f64) -> f64 {
    (f(x - h) - 2.0 * f(x) + f(x + h)) / h.powi(2)
}

fn third_derivative<F: Fn(f64) -> f64>(f: F, x: f64, h: f64) -> f64 {
    let h_half = h / 2.0;
    (f(x + 3.0 * h_half) - 3.0 * f(x + h_half) + 3.0 * f(x - h_half) - f(x - 3.0 * h_half)) / h.powi(3)
}

// Поиск оптимальных шагов
fn compute_optimal_hs(x: f64) -> (f64, f64, f64) {
    let interval = 0.1; // Окрестность точки x: [x - 0.1, x + 0.1]
    let a = x - interval;
    let b = x + interval;
    let step = 0.001;

    // M0 для E (погрешность округления)
    let m0 = find_max(f, a, b, step);
    let epsilon = f64::EPSILON;
    let e = m0 * epsilon;

    // M2, M3, M5 для оптимальных h
    let m2 = find_max(f_double_prime_exact, a, b, step);
    let m3 = find_max(|x| f_triple_prime_exact(x).abs(), a, b, step);
    let m5 = 1000.0;

    let h1_opt = 2.0 * (e / m2).sqrt();
    let h2_opt = (3.0 * e / m3).cbrt();
    let h3_opt = (45.0 * e / (4.0 * m5)).powf(0.2);

    (h1_opt, h2_opt, h3_opt)
}

// Поиск максимума функции на интервале
fn find_max<F: Fn(f64) -> f64>(f: F, a: f64, b: f64, step: f64) -> f64 {
    let mut max_val = f64::MIN;
    let mut x = a;
    while x <= b {
        let val = f(x).abs();
        if val > max_val {
            max_val = val;
        }
        x += step;
    }
    max_val
}

fn main() {
    let x = 2.0;

    // Точные значения производных (аналитические)
    let exact_f1 = f_prime_exact(x);
    let exact_f2 = f_double_prime_exact(x);
    let exact_f3 = f_triple_prime_exact(x);

    // Оптимальные шаги для каждой производной
    let (h1_opt, h2_opt, h3_opt) = compute_optimal_hs(x);

    // Приближенные значения производных
    let approx_f1_forward = forward_diff(f, x, h1_opt);
    let approx_f1_central = central_diff(f, x, h2_opt);
    let approx_f1_fourth = fourth_order_diff(f, x, h3_opt);

    let approx_f2 = second_derivative(f, x, h2_opt);
    let approx_f3 = third_derivative(f, x, h3_opt);

    // Вывод результатов
    println!("Точка x = {}\n", x);

    // Первая производная
    println!("Первая производная:");
    println!("Точное значение: {}", exact_f1);
    println!("Прямая разность (h = {:.2e}): {}", h1_opt, approx_f1_forward);
    println!("Центральная разность (h = {:.2e}): {}", h2_opt, approx_f1_central);
    println!("Формула 4-го порядка (h = {:.2e}): {}", h3_opt, approx_f1_fourth);
    println!("Абс. погрешности: {:.2e}, {:.2e}, {:.2e}",
        (exact_f1 - approx_f1_forward).abs(),
        (exact_f1 - approx_f1_central).abs(),
        (exact_f1 - approx_f1_fourth).abs()
    );

    // Вторая производная
    println!("\nВторая производная:");
    println!("Точное значение: {}", exact_f2);
    println!("Приближенное (h = {:.2e}): {}", h2_opt, approx_f2);
    println!("Абс. погрешность: {:.2e}", (exact_f2 - approx_f2).abs());

    // Третья производная
    println!("\nТретья производная:");
    println!("Точное значение: {}", exact_f3);
    println!("Приближенное (h = {:.2e}): {}", h3_opt, approx_f3);
    println!("Абс. погрешность: {:.2e}", (exact_f3 - approx_f3).abs());
}

Overwriting main.rs


In [8]:
!rustc main.rs && ./main

Точка x = 2

Первая производная:
Точное значение: -5.1407972157667094
Прямая разность (h = 8.92e-9): -5.140797309050553
Центральная разность (h = 4.74e-6): -5.140797215650708
Формула 4-го порядка (h = 2.76e-4): -5.1407972157693465
Абс. погрешности: 9.33e-8, 1.16e-10, 2.64e-12

Вторая производная:
Точное значение: -6.8022814144930335
Приближенное (h = 4.74e-6): -6.802237400687951
Абс. погрешность: 4.40e-5

Третья производная:
Точное значение: -3.4675914298712414
Приближенное (h = 2.76e-4): -3.4676882381925807
Абс. погрешность: 9.68e-5
